In [1]:
import os
import cv2
import pickle
import h5py
import numpy as np
import cv2
import glob
import pandas as pd

In [2]:
def preprocess_frame(frame, target_size=(200, 100)):
    """
    Preprocesses a frame for gaze prediction.
    Args:
    - frame: The input image frame (assumed to be in BGR format as per OpenCV standard)
    - target_size: The target size to which the frame should be resized (width, height)

    Returns:
    - Preprocessed frame
    """
    # Check if image is loaded correctly
    if frame is None:
        raise ValueError("Invalid input frame")

    # Convert to grayscale
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to get a binary image
    _, binary_image = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        raise ValueError("No contours found in the frame")

    # Find the largest contour based on area
    largest_contour = max(contours, key=cv2.contourArea)

    # Get the bounding box of the largest contour
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Crop the image using the bounding box
    cropped_image = frame[y:y+h, x:x+w]

    # Resize the cropped image to the target size
    resized_image = cv2.resize(cropped_image, target_size)
 
    # Convert to float and normalize
    preprocessed_image = resized_image.astype(np.float32) / 255.0

    return preprocessed_image

In [3]:
def get_h5_file_path(subdir):

    # Extract the parent directory of the 'subdir' to get the user's data directory
    user_data_dir = os.path.dirname(subdir)
    
    # Construct the path to the 'Calibration' directory
    calibration_dir = os.path.join(user_data_dir, 'Calibration')
    
    # Assuming there's only one .h5 file per user in the 'Calibration' directory
    for file in os.listdir(calibration_dir):
        if file.endswith('.h5'):
            if 'screenSize' in file:
                return os.path.join(calibration_dir, file)
    
    # If no .h5 file is found, return None or raise an error
    return None

In [4]:
def compute_global_stats_txt(base_dir):
    all_head_pose_data = []
    # Iterate through all text files recursively in the base directory
    for txt_file in glob.glob(os.path.join(base_dir, '**', '*.txt'), recursive=True):
        with open(txt_file, 'r') as file:
            for line in file:
                # Split the line into a list of values
                values = line.strip().split()
                # Extract head pose data (rotation and translation) from the line
                head_pose_data = [float(values[i]) for i in range(29, 35)] 
                all_head_pose_data.append(head_pose_data)

    all_head_pose_data = np.array(all_head_pose_data)
    # Compute minimum and maximum values for each column (dimension)
    min_vals = np.min(all_head_pose_data, axis=0)
    max_vals = np.max(all_head_pose_data, axis=0)

    return min_vals, max_vals

In [5]:
min_vals, max_vals = compute_global_stats_txt('../MPIIGaze/Data/original')

In [6]:
min_vals, max_vals

(array([-1.08600600e+00, -1.25604700e+00, -3.45308000e-01, -7.48378723e+02,
        -4.05566376e+02,  1.68924423e+02]),
 array([8.99462000e-01, 1.18543900e+00, 3.80730000e-01, 4.97009918e+02,
        3.27464874e+02, 1.73951575e+03]))

In [39]:
def read_h5(h5_file_path):
    with h5py.File(h5_file_path, 'r') as h5_file:
        h5_data = {key: h5_file[key][:] for key in h5_file.keys()}
    return h5_data

def normalize_annotations(annotation, width_pixel, height_pixel):
    normalized_x = float(annotation[0]) / width_pixel
    normalized_y = float(annotation[1]) / height_pixel
    return [normalized_x, normalized_y]

def normalize_head_pose(head_pose_data, min_vals, max_vals):
    normalized_head_pose_data = (head_pose_data - min_vals) / (max_vals - min_vals)
    return normalized_head_pose_data

def append_to_pickle(data, filename):
    try:
        with open(filename, 'rb') as file:
            existing_data = pickle.load(file)
    except FileNotFoundError:
        existing_data = {'X': [], 'Y': []}

    existing_data['X'].extend(data['X'])
    existing_data['Y'].extend(data['Y'])

    with open(filename, 'wb') as file:
        pickle.dump(existing_data, file)

def process_images_and_annotations(base_path, subdir_limit=1):
    X, Y= [], []
    batch_number = 0
    last_processed_dir = ""
    processed_dirs_count = 0
    # min_vals, max_vals = compute_global_stats_txt(base_path)

    try:
        with open('progress_log.txt', 'r') as file:
            batch_number, last_processed_dir = file.read().strip().split('\n')
            batch_number = int(batch_number)
    except FileNotFoundError:
        print("No progress log found. Starting from the beginning.")

    for subdir, dirs, files in os.walk(base_path):
        if processed_dirs_count >= subdir_limit:
            break  # Stop processing once the subdir limit is reached

        if subdir <= last_processed_dir:
            continue

        print(f"Processing directory {subdir}")

        if not any(file.endswith('.jpg') for file in files):
            continue

        processed_dirs_count += 1
        
        user = os.path.basename(os.path.dirname(subdir))
        h5_file_path = os.path.join(base_path, user, 'Calibration', 'screenSize.h5')
        
        if not os.path.exists(h5_file_path):
            print(f"No .h5 file found for user {user}. Skipping.")
            continue

        h5_data = read_h5(h5_file_path)
        width_pixel, height_pixel = h5_data['width_pixel'][0,0], h5_data['height_pixel'][0,0]

        annotation_file = os.path.join(subdir, 'annotation.txt')
        if not os.path.exists(annotation_file):
            print(f"No annotation file found for directory {subdir}. Skipping.")
            continue

        with open(annotation_file, 'r') as ann_file:
            annotations = [line.strip().split() for line in ann_file]

        for file in sorted(files):
            if file.endswith('.jpg'):
                image_path = os.path.join(subdir, file)
                image_index = sorted(files).index(file)
                if image_index < len(annotations):
                    annotation = normalize_annotations(annotations[image_index][24:26], width_pixel, height_pixel)
                    head_pose_data = np.array(annotations[image_index][29:35], dtype=float)
                    normalized_head_pose = normalize_head_pose(head_pose_data, min_vals, max_vals)
                    image = cv2.imread(image_path)
                    # Assuming preprocess_frame is a function you have defined elsewhere
                    preprocessed_image = preprocess_frame(image)
                    X.append(preprocessed_image)
                    # append both annotation and head pose data to Y
                    Y.append([annotation, normalized_head_pose])

        append_to_pickle({'X': X, 'Y': Y}, f'batches_head_pos/data_batch_{batch_number}.pkl')
        del X, Y, 
        X, Y = [], []
        batch_number += 1
        with open('progress_log.txt', 'w') as file:
            file.write(f"{batch_number}\n{subdir}")

    return X, Y, batch_number

In [48]:
# Load and preprocess the images and annotations
images, gaze_data, batch_number = process_images_and_annotations('../MPIIGaze/Data/Original',25)

In [30]:
#clear memory
import gc
del images
del gaze_data
gc.collect()

21